In [1]:
# import dependencies
import pandas as pd
import datetime
from dateutil import parser
import numpy as np


In [2]:
# read csv with data for open mic events
OMs = pd.read_csv('TCB_mics_calendar - TCB_mics_calendar copy 6.csv')

# remove rows that don't have a name or a venue
OMs = OMs.dropna(subset=['EVENT NAME', 'VENUE'])

# dictionary with adjusted column names
columns = {}

# list with columns to be dropped
drops = []

# adjust column names and determine which columns should be dropped
for col in OMs.columns:
    c = col.replace(' - MIC', '').replace('CATEGORIES', 'CATEGORY').replace('?', '').replace(' MICS', '').replace(' ', '_').lower()
    if '(dont_fill)' in c:
        drops.append(c)
    columns[col] = c

# set column names to adjusted column names
OMs = OMs.rename(columns=columns)

# remove columns that aren't supposed to be filled
OMs = OMs.drop(columns=drops)

OGs = OMs.copy()

# display dataframe
OMs


,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
0,(11AM-12PM Hourly) Improv Mic @ Fourth Wall Co...,Fourth Wall Comedy Cafe,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,1:45 PM,False,Open Mic,$5,No min.,NaN,True,True,For stand-up riffing and crowd work\nSuggestio...,5 min.,https://slotted.co/fwcafe
1,Improv Mic,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,12:00 PM,False,Improvised stand-up mic,$5,No min.,NaN,True,True,includes snacks and drinks\nSign-up in advance...,5 min.,https://slotted.co/thehollywoodcomedy
2,(1PM-6PM Hourly) The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,7:00 PM,False,Open Mic,$5,No min.,NaN,True,True,includes snacks & drinks\nSign-up in advance a...,5 min.,https://slotted.co/thehollywoodcomedy
3,(12PM-1PM Hourly) Flashback Mic (in Melrose),Flashback Mic (on Melrose),WeHo/Fairfax/Beverly Hills,2020-03-15,12:00 PM,2020-03-15,2:00 PM,False,Open Mic,$5,No min.,NaN,True,True,"*entrance located in rear, look for the red do...",5 min.,https://slotted.co/fbmic
4,(1PM-12AM Hourly) 15 Mins. The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,11:59 PM,False,Open Mic,$12,No min.,NaN,True,True,"\n\nin ""The Fancy Room"" (up front)\n$12 (inclu...",15 min.,https://slotted.co/thehollywoodcomedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Geeky Teas & Games,Geeky Teas & Games,Burbank/NoHo/Valley,2020-02-22,9:30 PM,2020-02-22,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes bottled water\nSign-up 9PM\nStart...,5 min.,NaN
457,Santa Monica Playhouse,Santa Monica Playhouse,Westside/Santa Monica,2020-02-29,9:30 PM,2020-02-29,12:00 AM,False,Open Mic,$5,No min.,NaN,True,True,"\n\nNot on Main Stage, in Other Theatre\nPassw...",5 min.,NaN
458,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,10:00 PM,2020-03-14,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
459,Atomic Wombat Comics,Atomic Wombat Comics,Orange County,2020-03-07,10:30 PM,2020-03-07,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/c...,5 min.,https://slotted.co/comedygrabbag


In [3]:
# list to hold time related columns
begin_and_end = ['start_date', 'end_date', 'start_time', 'end_time']

# convert data in these columns from strings to timestamps
for i , t in enumerate(begin_and_end):
    if i < 2:
        OMs[t] = OMs[t].apply(lambda x: parser.parse(x))
    else:
        OMs[t] = OMs[t].apply(lambda x: 60*parser.parse(x).hour + parser.parse(x).minute)
        
# sort events in chronological order
OMs = OMs.sort_values(['start_date', 'start_time'])


In [4]:
def time_string(minutes):
    
    hour = int(minutes/60)
    if hour > 12:
        if hour != 24:
            hour = f'{hour-12}:PM'
        else:
            hour = '12:AM'
    else:
        if hour != 12:
            hour = f'{hour}:AM'
        else: hour = f'12:PM'
            
    minute = minutes%60
    if minute == 0:
        hour = hour.replace(':', '')
    else:
        hour = hour.replace(':', f':{minute}')
    
    return hour



In [5]:
# OMs['new_start_time'] = 0
# OMs['new_end_time'] = 0

# # list to hold names of all unique events
# events = OMs['event_name'].unique()

# for e in events:
    
#     # all events that share a particular name
#     event = OMs.copy().loc[OMs['event_name'] == e,:]
    
#     # list to hold the indicies of events of a shared name
#     rows = list(event.index)

#     # consolidate times
#     new = 0
#     start = event['start_time'][rows[0]]
#     for i in range(1, len(rows)):
#         end = event[f'end_time'][rows[i]]
#         if event['start_time'][rows[i]] == event['end_time'][rows[i-1]]:
#             OMs.loc[rows[i-1: i+1], 'new_start_time'] = start
#             OMs.loc[rows[new:i+1], 'new_end_time'] = end
#         else:
#             new = i
#             start = event['start_time'][rows[i]]

#     # consolidate dates
#     new = 0
#     start = event['start_date'][rows[0]]
#     for i in range(1, len(rows)):
#         end = event[f'end_date'][rows[i]]
#         if (event['start_date'][rows[i]] - event['end_date'][rows[i-1]]).days < 2:
#             OMs.loc[rows[i-1: i+1], 'new_start_date'] = start
#             OMs.loc[rows[new:i+1], 'new_end_date'] = end
#         else:
#             new = i
#             start = event['start_date'][rows[i]]


In [6]:
OMs['new_start_time'] = OMs['start_time']
OMs['new_end_time'] = OMs['end_time']
OMs['new_start_date'] = OMs['start_date']
OMs['new_end_date'] = OMs['end_date']

# # list to hold names of all unique events
# events = OMs['event_name'].unique()

# for e in events:

# all events that share a particular name
event = OMs.copy().loc[OMs['event_name'] == 'Echoes on Pico',:]

# list to hold the indicies of events of a shared name
rows = list(event.index)

# consolidate times
new = 0
start = event['start_time'][rows[0]]
for i in range(1, len(rows)):


    old_end_time = event['end_time'][rows[i-1]]
    new_start_time = event['start_time'][rows[i]]

    old_start_date = event['start_date'][rows[i-1]]
    new_start_date = event['start_date'][rows[i]]

    end = event[f'end_time'][rows[i]]

    if old_end_time == new_start_time and old_start_date == new_start_date:
        OMs.loc[rows[i-1: i+1], 'new_start_time'] = start
        OMs.loc[rows[new:i+1], 'new_end_time'] = end
        event.loc[rows[i-1: i+1], 'new_start_time'] = start
        event.loc[rows[new:i+1], 'new_end_time'] = end
    else:
        new = i
        start = event['start_time'][rows[i]]

# consolidate dates
new = 0
start = event['start_date'][rows[0]]
for i in range(1, len(rows)):

    old_start_time = event['new_start_time'][rows[i-1]]
    old_end_time = event['new_end_time'][rows[i-1]]
    new_start_time = event['new_start_time'][rows[i]]
    new_end_time = event['new_start_time'][rows[i]]

    old_end_date = event['end_date'][rows[i-1]]
    new_start_date = event['start_date'][rows[i]]


    end = event[f'end_date'][rows[i]]
    
    print((new_start_date - old_end_date).days < 2)
    print(old_start_time, new_start_time)
    print(new_end_time, old_end_time)
    print('\n')
    


    if (new_start_date - old_end_date).days < 2 and old_start_time == new_start_time:
        OMs.loc[rows[i-1: i+1], 'new_start_date'] = start
        OMs.loc[rows[new:i+1], 'new_end_date'] = end
    else:
        new = i
        start = event['start_date'][rows[i]]


True
1200 1200
1200 1380


True
1200 1200
1200 1380


False
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380


True
1200 1200
1200 1380




In [7]:
OMs.loc[OMs['event_name'] == 'Echoes on Pico',:]

,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,...,free,show_map_link,show_map,event_description,stage_time,event_website,new_start_time,new_end_time,new_start_date,new_end_date
443,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,1200,2020-03-14,1260,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-14,2020-03-14
454,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,1260,2020-03-14,1320,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-14,2020-03-14
458,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,1320,2020-03-14,1380,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-14,2020-03-14
104,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,1200,2020-03-16,1260,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-16,2020-03-20
111,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,1260,2020-03-16,1320,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-16,2020-03-20
116,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,1320,2020-03-16,1380,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-16,2020-03-20
168,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,1200,2020-03-17,1260,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-16,2020-03-20
191,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,1260,2020-03-17,1320,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-16,2020-03-20
195,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,1320,2020-03-17,1380,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-16,2020-03-20
243,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-18,1200,2020-03-18,1260,False,Open Mic,$5,...,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/,1200,1380,2020-03-16,2020-03-20


In [8]:
OMs[['new_start_time', 'new_end_time']] = OMs[['new_start_time', 'new_end_time']].astype('int32')

In [9]:
OMs['event_description'] = OMs.apply(lambda x: x['event_description'].replace(time_string(x['start_time']), time_string(x['new_start_time'])), axis=1)
OMs['event_description'] = OMs.apply(lambda x: x['event_description'].replace(time_string(x['end_time']), time_string(x['new_end_time'])), axis=1)



In [10]:
for bne in begin_and_end:
    
    OMs = OMs.drop(columns=bne)
    OMs = OMs.rename(columns={'new_' + bne: bne})

OMs['start_time'] = OMs['start_time'].apply(time_string)
OMs['end_time'] = OMs['end_time'].apply(time_string)

# drop rows with the same information
OMs = OMs.drop_duplicates()



In [11]:
OGs.loc[OGs['event_name'] == 'Echoes on Pico',:].sort_values(['start_date', 'start_time'])

,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
458,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,10:00 PM,2020-03-14,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
443,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,8:00 PM,2020-03-14,9:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
454,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,9:00 PM,2020-03-14,10:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
116,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,10:00 PM,2020-03-16,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
104,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,8:00 PM,2020-03-16,9:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
111,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,9:00 PM,2020-03-16,10:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
195,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,10:00 PM,2020-03-17,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
168,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,8:00 PM,2020-03-17,9:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
191,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,9:00 PM,2020-03-17,10:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
270,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-18,10:00 PM,2020-03-18,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/


In [12]:
# display final dataframe
OMs


,event_name,venue,neighborhood,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website,start_time,end_time,start_date,end_date
90,Storytelling Mic,LA Wine,DTLA,False,Storytelling,$0,No min.,Free mics\n,True,True,\n\nin Suite 200\nFor storytelling only\nSign-...,5 min.,NaN,7:30PM,9:30PM,2020-01-13,2020-01-13
303,LA Connection Comedy Theater,LA Connection Comedy Theater,Burbank/NoHo/Valley,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/h...,5 min.,https://slotted.co/hotmedusa-,6PM,7PM,2020-01-16,2020-01-16
264,The Barkley,The Barkley,Pasadena,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nStarts 9PM\nRolling lottery\nHosted by Van...,3 min.,NaN,9PM,11PM,2020-01-22,2020-01-22
383,(7-9PM Hourly) The Open Mic Spot,The Open Mic Spot,South LA,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://openmicspot....,5 min.,https://openmicspot.com/open-mic/,7PM,9PM,2020-01-24,2020-01-24
447,(8-10PM Hourly) Therapeutic Noise (in Long Beach),Therapeutic Noise,Long Beach,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/t...,6 min.,https://slotted.co/therapeuticnoise,8PM,11PM,2020-01-25,2020-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Edendale Branch Library,Edendale Branch Library,Los Feliz/Silverlake/Echo Park,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nSign-up 5:45PM\nStarts 6PM\nEnds 7:30PM\nF...,5 min.,NaN,6PM,8PM,2020-04-02,2020-04-02
368,Improv Jam,UCB Sunset Inner Sanctum,Hollywood/East Hollywood,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nFor improv only\nStarts 6PM\nAnyone can play,NaN,NaN,6PM,7PM,2020-04-03,2020-04-03
290,Lyric Hyperion Theatre & Cafe,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nSign-up 4:30PM \nStarts 5PM \nEnds 7PM \nL...,4 min.,NaN,4PM,6PM,2020-04-04,2020-04-04
71,Character Spa,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,False,Character Mic,$0,No min.,Free mics\n,True,True,\n\nFor characters only\nThis mic is an opport...,7 min.,https://slotted.co/characterspa,4PM,6PM,2020-04-06,2020-04-06


In [13]:
len(OMs)

438

In [14]:
events = []

for e in OMs['event_name'].unique():
    event = OMs.loc[OMs['event_name'] == e,:]
    if len(event) > 1:
        events.append(e)
        

In [15]:
print(len(events))
events

69


['The Barkley',
 '(8-10PM Hourly) Therapeutic Noise (in Long Beach)',
 '(7-9PM Hourly) Therapeutic Noise (in Long Beach)',
 'Silverlake Lounge',
 'Universal Bar & Grill',
 'The Rec Room',
 'Hollywood Improv Comedy Lab',
 'Improv Jam',
 'HaHa Cafe',
 'TREE4ORT',
 'The Clubhouse',
 'Sycamore Tavern',
 '(12-10PM Hourly) Flashback Mic (in Melrose)',
 'Geeky Teas & Games',
 'Tribal Cafe',
 'The Open Mic Spot',
 '(8PM-12AM Hourly) The Hollywood Comedy',
 'Atomic Wombat Comics',
 'The Moth ',
 'Tao Comedy Studio',
 '(3PM-4PM Hourly) 10 Mins. The Comedy Office (Unit #3 Upstairs)',
 "(3PM-10PM Hourly) Bert's Back Room",
 '(4PM-7PM Hourly) The Comedy Office (Unit #3 Upstairs)',
 'The Green Room',
 'The Ruby LA',
 '(1PM-6PM Hourly) The Hollywood Comedy',
 '(1PM-12AM Hourly) 15 Mins. The Hollywood Comedy',
 'ACME Comedy ',
 'Improv Mic',
 'LTE VIsions',
 "(8PM-10PM Hourly) 10 Min Spots Bert's Back Room Lounge",
 'Flappers Bar',
 '(3PM-12AM Hourly) 10 Mins. The Hollywood Comedy',
 "(4PM-10PM Hourly

In [16]:
event = OMs.loc[OMs['event_name'] == 'Garage behind Food 4 Less',:]
print(i)
print(events[i]+'\n')
for d in event['event_description']:
    print(d)

event

17
Atomic Wombat Comics



in alley behind Food 4 Less
Starts 7PM
Ends 9:30PM
First come, first served
6 min.
Best set of the night wins $10
Hosted by Victor Martinez Jr.


in alley behind Food 4 Less
Starts 11PM
Ends 2AM
First come, first served
6 min.
Hosted by Victor Martinez Jr.


,event_name,venue,neighborhood,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website,start_time,end_time,start_date,end_date
378,Garage behind Food 4 Less,Garage behind Food 4 Less,Koreatown/Mid-City,False,Open Mic,$2,No min.,NaN,True,True,\n\nin alley behind Food 4 Less\nStarts 7PM\nE...,6 min.,NaN,7PM,9:30PM,2020-03-20,2020-03-20
455,Garage behind Food 4 Less,Garage behind Food 4 Less,Koreatown/Mid-City,False,Open Mic,$0,No min.,NaN,True,True,\n\nin alley behind Food 4 Less\nStarts 11PM\n...,6 min.,NaN,9PM,2AM,2020-03-21,2020-03-21


In [17]:
sus = ['Hollywood Improv Comedy Lab', 'HaHa Cafe', 'TREE4ORT', 'Atomic Wombat Comics', "(3PM-10PM Hourly) Bert's Back Room", '(4PM-7PM Hourly) The Comedy Office (Unit #3 Upstairs)', 'The Green Room', '(1PM-6PM Hourly) The Hollywood Comedy', 'Improv Mic', 'Flappers Bar', 'The Harp Inn Irish Pub', '(11AM-12PM Hourly) Improv Mic @ Fourth Wall Comedy Cafe', '(12PM-1PM Hourly) Flashback Mic (in Melrose)', 'Improv Mic @ Flashback Mic (in Melrose)', 'Improv Mic @ Flashback Mic (in Melrose)', 'Improv Mic @ NoHo Fourth Wall', '(5PM-10PM Hourly) NoHo Fourth Wall', 'Garage behind Food 4 Less']



In [18]:
i = 4

In [19]:

event = OMs.loc[OMs['event_name'] == sus[i],:]
print(i)
print(sus[i]+'\n')
for d in event['event_description']:
    print(d)
i += 1


event

4
(3PM-10PM Hourly) Bert's Back Room



Upstairs, entrance through door in between Von Dutch and ikon
Sign-up in advance at https://slotted.co/bertsbackroom
Mic runs every hour on the hour from 3PM-10PM (venue closes 11PM)
5 min.
Wait list for walk-ins and drop outs will be first come, first served
Hosted by Rachel Cuthbert


Upstairs, entrance through door in between Von Dutch and ikon 
Sign-up in advance at https://slotted.co/bertsbackroom
Mic runs every hour on the hour from 3PM-10PM (venue closes 11PM)
5 min.
Wait list for walk-ins and drop outs will be first come, first served
Hosted by Rachel Cuthbert


Upstairs, entrance through door in between Von Dutch and ikon 
Sign-up in advance at https://slotted.co/bertsbackroom
Mic runs every hour on the hour from 3PM-10PM (venue closes 11PM)
5 min.
Wait list for walk-ins and drop outs will be first come, first served
Hosted by Rachel Cuthbert


Upstairs, entrance through door in between Von Dutch and ikon 
Sign-up in advance at https://s

,event_name,venue,neighborhood,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website,start_time,end_time,start_date,end_date
420,(3PM-10PM Hourly) Bert's Back Room,Bert's Back Room,WeHo/Fairfax/Beverly Hills,False,Open Mic,$5,No min.,NaN,True,True,"\n\nUpstairs, entrance through door in between...",5 min.,https://slotted.co/bertsbackroom,3PM,11PM,2020-03-07,2020-03-07
11,(3PM-10PM Hourly) Bert's Back Room,Bert's Back Room,WeHo/Fairfax/Beverly Hills,False,Open Mic,$5,No min.,NaN,True,True,"\n\nUpstairs, entrance through door in between...",5 min.,https://slotted.co/bertsbackroom,3PM,11PM,2020-03-08,2020-03-08
63,(3PM-10PM Hourly) Bert's Back Room,Bert's Back Room,WeHo/Fairfax/Beverly Hills,False,Open Mic,$5,No min.,NaN,True,True,"\n\nUpstairs, entrance through door in between...",5 min.,https://slotted.co/bertsbackroom,3PM,11PM,2020-03-09,2020-03-09
136,(3PM-10PM Hourly) Bert's Back Room,Bert's Back Room,WeHo/Fairfax/Beverly Hills,False,Open Mic,$5,No min.,NaN,True,True,"\n\nUpstairs, entrance through door in between...",5 min.,https://slotted.co/bertsbackroom,3PM,11PM,2020-03-10,2020-03-10
210,(3PM-10PM Hourly) Bert's Back Room,Bert's Back Room,WeHo/Fairfax/Beverly Hills,False,Open Mic,$5,No min.,NaN,True,True,"\n\nUpstairs, entrance through door in between...",5 min.,https://slotted.co/bertsbackroom,3PM,11PM,2020-03-11,2020-03-11
283,(3PM-10PM Hourly) Bert's Back Room,Bert's Back Room,WeHo/Fairfax/Beverly Hills,False,Open Mic,$5,No min.,NaN,True,True,"\n\nUpstairs, entrance through door in between...",5 min.,https://slotted.co/bertsbackroom,3PM,11PM,2020-03-12,2020-03-12
356,(3PM-10PM Hourly) Bert's Back Room,Bert's Back Room,WeHo/Fairfax/Beverly Hills,False,Open Mic,$5,No min.,NaN,True,True,"\n\nUpstairs, entrance through door in between...",5 min.,https://slotted.co/bertsbackroom,3PM,11PM,2020-03-13,2020-03-13


In [22]:
event['event_description'].unique()

array(['\n\nSign-up 9:30PM\nLottery\n3 min.',
       '\n\nStarts 8:30PM\nLottery\n3 min.',
       '\n\nStarts 10:30PM\nLottery \n3 min.\nHosted by Mike Perkins',
       '\n\nSign-up 11PM\nStarts 11:30PM\nLottery\n3 min.',
       '\n\nStarts 11PM\nLottery\n3 min.'], dtype=object)

In [20]:
super_sus = ['Atomic Wombat Comics', 'The Green Room']

noi = [425,2,56,408,1]

In [21]:
event = OGs.loc[OGs['event_name'] == 'Flappers Bar',:]

event.sort_values(['start_date', 'start_time'])



,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
50,Flappers Bar,Flappers Comedy Club,Burbank/NoHo/Valley,2020-03-08,9:30 PM,2020-03-08,11:30 PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nSign-up 9:30PM\nLottery\n3 min.,3 min.,NaN
105,Flappers Bar,Flappers Comedy Club,Burbank/NoHo/Valley,2020-03-09,8:30 PM,2020-03-09,10:30 PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nStarts 8:30PM\nLottery\n3 min.,3 min.,NaN
187,Flappers Bar,Flappers Comedy Club,Burbank/NoHo/Valley,2020-03-10,8:30 PM,2020-03-10,10:30 PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nStarts 8:30PM\nLottery\n3 min.,3 min.,NaN
263,Flappers Bar,Flappers Comedy Club,Burbank/NoHo/Valley,2020-03-11,8:30 PM,2020-03-11,10:30 PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nStarts 8:30PM\nLottery\n3 min.,3 min.,NaN
343,Flappers Bar,Flappers Comedy Club,Burbank/NoHo/Valley,2020-03-12,10:30 PM,2020-03-12,11:59 PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nStarts 10:30PM\nLottery \n3 min.\nHosted b...,3 min.,NaN
404,Flappers Bar,Flappers Comedy Club,Burbank/NoHo/Valley,2020-03-13,10:30 PM,2020-03-13,11:59 PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nSign-up 11PM\nStarts 11:30PM\nLottery\n3 min.,3 min.,NaN
460,Flappers Bar,Flappers Comedy Club,Burbank/NoHo/Valley,2020-03-14,11:00 PM,2020-03-14,1:30 AM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nStarts 11PM\nLottery\n3 min.,3 min.,NaN
